In [ ]:
import numpy as np
import os
import pandas as pd
from sklearn import svm
import time

geno = np.loadtxt('/datasets/cs284-sp21-A00-public/ps2/precomputed/ps2_pca.genotypes.tab') # (28622 SNPs, 2504 people)
pop_6 = np.load('pop_6.npy')

In [ ]:
train = geno[:,:2000]
test = geno[:,2000:]
train_Y = pop_6[:2000]
test_Y = pop_6[2000:]

数据集0.8/0.2划分，可以尝试些别的分类算法。直接在整个数据集上run的时间非常长

In [ ]:
def simple_SVM(X,Y,test_X,test_Y):
    time0=time.time()
    
    clf = svm.SVC(kernel='linear')
    clf.fit(X,Y)
    
    print('Train time:',time.time()-time0)
    
    print('train acc:',clf.score(X, Y))
    print('test acc:',clf.score(test_X, test_Y))

In [ ]:
simple_SVM(train.T,train_Y,test.T,test_Y)